In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd
 
server = SSHTunnelForwarder(
    ssh_address_or_host=('10.12.0.8', 2043),
    ssh_username='root', 
    ssh_password = 'ynby20250625', 
 
    
    remote_bind_address=('localhost' , 5432)) 
server.start()
conn = psycopg2.connect(database = 'FAERS20-24', 
                            user = 'postgres',     
                            password = 'woshipostgres', 
                            host = '127.0.0.1',   
                            port = server.local_bind_port)

In [26]:
import math

In [ ]:
def get_drug_reaction(drugid, con = conn):
    sql = (f"SELECT DISTINCT reactid FROM cdir_fo WHERE drugbank_id = '{drugid}';")
    drug_reaction = pd.read_sql(sql, con = conn)
    #print(drug_reaction)
    reactids = drug_reaction.iloc[:, 0]
    return reactids

def count_drug(drugid, con = conn):
    '''
    Figure out the value 'a+c'(the sum of recordings of all reactions 
    triggered by the given drug) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM drugcount WHERE drugbank_id = '{drugid}';")
    drugcount = pd.read_sql(sql, con=conn)
    return drugcount['count'].values[0]

def count_react(reactid, con = conn):
    '''
    Figure out the value 'a+b'(the sum of recordings of the given reaction 
    ) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM reactcount WHERE reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]

def count_a(drugid, reactid, con = conn):
    '''
    Figure out the value 'a' (the count of recordings associating 
    with given drug and focusing adverse reaction) 
    in the Disproportional Method
    '''
    sql = (f"SELECT count(*) FROM cdir_fo WHERE drugbank_id = '{drugid}' AND reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]

def ror_ci(a, b, c, d, ror):
    ror_upper_ci_power = math.log(ror) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ror_lower_ci_power = math.log(ror) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ror_upper_ci = math.exp(ror_upper_ci_power)
    ror_lower_ci = math.exp(ror_lower_ci_power)
    return ror_lower_ci, ror_upper_ci


def prr_ci(a, b, c, d, prr):
    prr_upper_ci_power = math.log(prr) + 1.96 * math.sqrt(1/a + 1/b - 1/(a + c) - 1/(b + d))
    prr_lower_ci_power = math.log(prr) - 1.96 * math.sqrt(1/a + 1/b - 1/(a + c) - 1/(b + d))
    prr_upper_ci = math.exp(prr_upper_ci_power)
    prr_lower_ci = math.sqrt(prr_lower_ci_power)
    return prr_lower_ci, prr_upper_ci

In [ ]:
def calculate(con = conn):
    total = 6981059
    sql = ("SELECT DISTINCT drugbank_id FROM drug21_24_map ORDER BY drugbank_id ASC limit 4")
    drugids = pd.read_sql(sql, con = conn)
    for drugid in drugids['drugbank_id']:
        print(drugid)
        reactids = get_drug_reaction(drugid)
        ac = count_drug(drugid)
        for reactid in reactids:
            ab = count_react(reactid)
            a = count_a(drugid, reactid)
            b = ab - a
            c = ac - a
            d = total - ab - ac + a
            if b == 0 or c == 0:
                a+=0.5
                b+=0.5
                c+=0.5
                d+=0.5
            ror = (a * d) / (b * c)
            ror_lower_ci, ror_upper_ci = ror_ci(a, b, c, d, ror)

            prr = a * (b + d)/(b * ac)
            prr_lower_ci, prr_upper_ci = prr_ci(a, b, c, d, prr)

            return drugid, reactid, a, b, c, d, ror, ror_lower_ci, ror_upper_ci, prr, prr_lower_ci, prr_upper_ci


In [ ]:
calculate()

 LYSINE AMIDOTRIZOATE
reaction_id: 140214, a: 2.5, b: 162207.5, c: 0.5, d: 6818850.5, ror: 210.18912504045744
reaction_id: 196523, a: 2.5, b: 254721.5, c: 0.5, d: 6726336.5, ror: 132.03315189334234
reaction_id: 24134, a: 2.5, b: 21131.5, c: 0.5, d: 6959926.5, ror: 1646.8131699122164
reaction_id: 31317, a: 2.5, b: 30018.5, c: 0.5, d: 6951039.5, ror: 1157.7926112230791
reaction_id: 35622668, a: 2.5, b: 42236.5, c: 0.5, d: 6938821.5, ror: 821.4247747801072
reaction_id: 4010894, a: 2.5, b: 159531.5, c: 0.5, d: 6821526.5, ror: 213.79873253871492
reaction_id: 4031840, a: 2.5, b: 2533.5, c: 0.5, d: 6978524.5, ror: 13772.497533057036
reaction_id: 4152347, a: 2.5, b: 66647.5, c: 0.5, d: 6914410.5, ror: 518.7299223526763
reaction_id: 4161682, a: 2.5, b: 52335.5, c: 0.5, d: 6928722.5, ror: 661.9524510131746
reaction_id: 4196193, a: 2.5, b: 63571.5, c: 0.5, d: 6917486.5, ror: 544.0713605939768
reaction_id: 4213725, a: 2.5, b: 9872.5, c: 0.5, d: 6971185.5, ror: 3530.608002025829
reaction_id: 422393

In [ ]:
DB00945

In [25]:
drugid = 'DB00945'
total = 6981059
print(drugid)
reactids = get_drug_reaction(drugid)
ac = count_drug(drugid)
for reactid in reactids:
    ab = count_react(reactid)
    a = count_a(drugid, reactid)
    b = ab - a
    c = ac - a
    d = total - ab - ac + a
    if b == 0 or c == 0:
        a+=0.5
        b+=0.5
        c+=0.5
        d+=0.5
    ror = (a * d) / (b * c)
    print(f'reaction_id: {reactid}, a: {a}, b: {b}, c: {c}, d: {d}, ror: {ror}')

DB00945
reaction_id: 4332304, a: 119, b: 3005, c: 221292, d: 6756643, ror: 1.2091153757649487
reaction_id: spontaneous heparin-induced thrombocytopenia syndrome, a: 4, b: 8, c: 221407, d: 6759640, ror: 15.265190350801916
reaction_id: 444100, a: 68, b: 2956, c: 221343, d: 6756692, ror: 0.7022193837658887
reaction_id: 374366, a: 64, b: 871, c: 221347, d: 6758777, ror: 2.243656129908748
reaction_id: 40342419, a: 3, b: 94, c: 221408, d: 6759554, ror: 0.9743570548874053
reaction_id: ear haemorrhage, a: 33, b: 682, c: 221378, d: 6758966, ror: 1.4773226875998693
reaction_id: csf test abnormal, a: 5, b: 93, c: 221406, d: 6759555, ror: 1.6414050905751016
reaction_id: 4332369, a: 9, b: 104, c: 221402, d: 6759544, ror: 2.6420743193897906
reaction_id: vascular operation, a: 5, b: 104, c: 221406, d: 6759544, ror: 1.4677925481833236
reaction_id: 4234997, a: 127, b: 1687, c: 221284, d: 6757961, ror: 2.2990811792449963
reaction_id: corynebacterium test positive, a: 1, b: 12, c: 221410, d: 6759636, ror

KeyboardInterrupt: 